## Embedding model ##
>Floris Menninga \
>Datum: 8-01-2026

In [2]:
# %pip install tensorflow keras
# %pip install tqdm
# %pip install tensorflow_text
#%pip install bs4
#%pip install joblib

from data_parser import xml_parser
import EmbeddingModel

import io
import re
import string
import tqdm

import numpy as np
import requests
import tensorflow as tf
import tensorflow_text as tf_text
import joblib
import tensorflow as tf
from tensorflow.keras import layers
import lxml

%load_ext tensorboard
AUTOTUNE = tf.data.AUTOTUNE
SEED = 0

2026-01-19 15:57:38.636175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768834659.193557   32357 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768834659.397676   32357 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768834659.770693   32357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768834659.770719   32357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768834659.770722   32357 computation_placer.cc:177] computation placer alr

In [3]:
print("Gpu's: ", len(tf.config.list_physical_devices('GPU')))

Gpu's:  1


## Tokenizer

De eerste stap is het omzetten van de text in tokens:
In tegenstelling to methodes zoals bytepair encoding etc. worden de tokens gemaakt door de zinnen \
te splitten op spaties. 

In [4]:
sentence = "Hallo, dit is een testzin om het programma te testen."
tokens = list(sentence.lower().split())
print(len(tokens))

10


### Vocabulary maken van deze tokens:

In [5]:
vocab, index = {}, 1 
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)

print(vocab)

{'<pad>': 0, 'hallo,': 1, 'dit': 2, 'is': 3, 'een': 4, 'testzin': 5, 'om': 6, 'het': 7, 'programma': 8, 'te': 9, 'testen.': 10}


### Inverse vocabulary:
Van integer index naar token:

In [5]:
inverse_vocab = {index: token for token, index in vocab.items()}

print(inverse_vocab)

{0: '<pad>', 1: 'hallo,', 2: 'dit', 3: 'is', 4: 'een', 5: 'testzin', 6: 'om', 7: 'het', 8: 'programma', 9: 'te', 10: 'testen.'}


## Vectorizeren van de zin:

In [6]:
test_sequence = [vocab[word] for word in tokens]

print(test_sequence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Skip-gram maken:
Het embedding model zal gemaakt worden met behulp van het Word2Vec algorithme. Verdere uitleg over hoe dit algorithme werkt komt straks maar om dit te kunnen gebruiken 
moet er eerst een Skip-gram model gemaatk worden. 
Het belangrijkste punt van een skip-gram is dat de vector representatie gebaseerd is op de ruimtelijke nabijheid van woorden in een zin. 
Als het woord "Ketting" altijd gevolgd zou worden door het woord "zaag" zal deze combinatie van woorden vaker voorkomen dan in andere combinaties.
Een skip-gram model is een klein neural network met een inputlayer, embedding layer en output layer. 
Dit model moet een waarschijnlijkheids verdelings vector geven voor een gegeven input woord. Dat is dus de kans dat deze twee woorden samen voorkomen binnen de context lengte waarmee het model getrained is. De som van deze waarschijnlijkheids verdeling is 1.

In [7]:
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      test_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=0
)

print(len(positive_skip_grams))

34


In [8]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(3, 4): (is, een)
(1, 2): (hallo,, dit)
(8, 9): (programma, te)
(6, 8): (om, programma)
(4, 3): (een, is)


In [9]:
target_word, context_word = positive_skip_grams[0]

num_ns = 10

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


tf.Tensor([ 6  1  5  0  8  9  7  2 10  4], shape=(10,), dtype=int64)
['om', 'hallo,', 'testzin', '<pad>', 'programma', 'te', 'het', 'dit', 'testen.', 'een']


I0000 00:00:1768085204.341126   91755 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4649 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [10]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word


In [11]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")


target_index    : 3
target_word     : is
context_indices : [ 4  6  1  5  0  8  9  7  2 10  4]
context_words   : ['een', 'om', 'hallo,', 'testzin', '<pad>', 'programma', 'te', 'het', 'dit', 'testen.', 'een']
label           : [1 0 0 0 0 0 0 0 0 0 0]


In [12]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)


[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=10,
          seed=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=0,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels



## Word2Vec:


Hier zal de dataset ingeladen worden:

In [ ]:
#text_ds = tf.data.TextLineDataset(["/run/media/floris/FLORIS_3/Data_set/PubMed/dataset_100914.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds = tf.data.TextLineDataset(["/home/floris/Downloads/trainingsData.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [15]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
sequence_length = 5
vocab_size = 25000

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [16]:
vectorize_layer.adapt(text_ds.batch(512))

2026-01-10 23:46:51.390358: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', np.str_('the'), np.str_('of'), np.str_('and'), np.str_('in'), np.str_('to'), np.str_('a'), np.str_('with'), np.str_('for'), np.str_('was'), np.str_('were'), np.str_('that'), np.str_('cancer'), np.str_('is'), np.str_('as'), np.str_('by'), np.str_('cells'), np.str_('patients'), np.str_('or')]


In [18]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(512).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [19]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

46619


2026-01-10 23:46:58.771098: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
# for seq in sequences[:5]:
#   print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

In [21]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=5,
    num_ns=3,
    vocab_size=vocab_size,
    seed=0)

print(f"\ntargets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

Generating Positive Pairs: 100%|██████████| 46619/46619 [00:00<00:00, 58385.27it/s]



targets.shape: (160317,)
contexts.shape: (160317, 4)
labels.shape: (160317, 4)


Gegenereerde data opslaan:
Het duurde lang om te genereren dus sla ik het op in een .joblib bestand, dit zal ik ook voor sommige vervolg stappen doen.


In [22]:
# joblib_list = [targets, contexts, labels]

# joblib.dump(joblib_list, "targets_contexts_labels.joblib")

In [23]:
BATCH_SIZE = 20
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(20,), dtype=tf.int32, name=None), TensorSpec(shape=(20, 4), dtype=tf.int32, name=None)), TensorSpec(shape=(20, 4), dtype=tf.int64, name=None))>


In [24]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [25]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 8
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [27]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [28]:
word2vec.fit(dataset, epochs=13, callbacks=[tensorboard_callback])

Epoch 1/13


I0000 00:00:1768085220.663878   91872 service.cc:152] XLA service 0x7f8bbc002140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768085220.663893   91872 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2026-01-10 23:47:00.677880: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1768085220.724569   91872 cuda_dnn.cc:529] Loaded cuDNN version 90300


  72/8015 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.2302 - loss: 1.3866 

I0000 00:00:1768085220.993679   91872 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8015/8015 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.7878 - loss: 0.7305
Epoch 2/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9032 - loss: 0.2976
Epoch 3/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9148 - loss: 0.2371
Epoch 4/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9307 - loss: 0.1932
Epoch 5/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9503 - loss: 0.1442
Epoch 6/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9691 - loss: 0.0949
Epoch 7/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9825 - loss: 0.0553
Epoch 8/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9912 - loss: 0.0291
Epoch 9/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9957 - loss: 0.0139
Epoch 10/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9985 - loss: 0.0060
Epoch 11/13
8015/8015 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9994 - loss: 0.0024
Epoch 12/13
8015/8015 ━━━━━━━━

In [29]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
num_tokens = min(len(vocab), len(weights))

with io.open('vectors.tsv', 'w', encoding='utf-8') as out_v, \
     io.open('metadata.tsv', 'w', encoding='utf-8') as out_m:

    # Skip de padding token index 0:
    for index in range(1, num_tokens):
        word = vocab[index]
        vec = weights[index]
        
        clean_word = word.strip()
        
        if not clean_word:
            clean_word = "[UNK_EMPTY]"

        # Vector
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        
        # Metadata
        out_m.write(clean_word + "\n")

print(f"{num_tokens - 1} vectors en metadata entries.")

Saved 24999 vectors and metadata entries.


## Trainen van het model:

In [ ]:
model = EmbeddingModel.EmbeddingModel()
model.load_dataset()
model.create_vocab()
model.vectorize()
model.generate_trainingdata()
model.word2vec()
model.fit()
model.save2file()

## Data:
Als trainingsdata gebruik ik 869597 Pubmed artikelen (~70GB) die ik gedownload heb van https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/

oa_comm_xml.PMC000xxxxxx.baseline.2025-06-26.tar.gz t/m oa_comm_xml.PMC004xxxxxx.baseline.2025-06-26.tar.gz

Deze zijn nieuwer dan de artikelen uit 2021 die op /commons/data/NCBI/PubMed/ staan.

Vervolgens moeten deze artikelen doorzocht worden op inhoud. \
De .xml bestanden hebben headers voor titel, abstract, body etc. deze doorzoek ik met behulp van mijn klasse: data_parser.py

Het filteren op de drie keywords "cancer", "melanoma" en "carcinoma" duurde 9 uur en 30 minuten en reduceerde de hoeveelheid artikelen 
van 869597 naar 100914 (2.7GB).

100914 Artikelen met deze zoekcriteria opgeslagen in: /run/media/floris/FLORIS_3/Data_set/PubMed/dataset_large.txt

In [32]:
keyword_list = ["cancer", "melanoma", "carcinoma"] # Niet hoofdlettergevoelig...

trainings_data = xml_parser("/run/media/floris/FLORIS_3/Data_set/PubMed/PMC000xxxxxx/", "/home/floris/Downloads/trainingsData3.txt", keyword_list)

#trainings_data.run()



## Resultaten:

Het eerste model was met de volgende hyperparameters getrained:

vocabulary grootte: 20000 \
context lengte: 100 \
window_size=5 \
num_ns=4
dims = 128

Het model was erg overfit op de trainingsdata, de loss was bijna 0 en accuratesse bijna 1. \
De volgende poging zal een minder grote vocabulary krijgen en minder dimenties om beter te generaliseren i.p.v. de trainingsdata te onthouden. 

De onderstaande afbeelding weergeeft de loss en accuracy grafieken met Tensorboard:

![](img/run_1.png)


Tweede run:

